# Convolución de matrices para procesado de imágenes
> Con un enfoque al paradigma funcional de programación en python.

- toc: false
- badges: true
- comments: true
- categories: [python, jupyter, functional programming]
- image: images/name.ext

![](../images/convolutionexample.png)

Una de las aplicaciones más importantes de la convolución de matrices es el procesamiento de imágenes, mediante un proceso iterativo que recorre cada pixel de una imagen, obteniendo valores de los pixeles circundantes y aplicando una transformación a estos datos, se puede obtener información relevante y realzar o mejorar aspectos que no son claramente notorios a simple vista o que son difíciles de procesar sin un tratamiento previo.

En formato digital, una imagen de mapa de bits tiene la estructura de una matriz de dos dimensiones, puede representarse como una función $f(x,y)$ donde $x$ e $y$ nos dan la posición de cada pixel y la función nos devuelve su 'color'. Para visualizar el color de este pixel, se usa un vector de tres valores que representan la intensidad de los colores rojo, verde y azul, cada uno tomando valores desde 0 a 255.

Entonces, la convolución de matrices consiste en la sumatoria de la multiplicación de cada elemento de un kernel predefinido con cada elemento de una sub matriz (sección de una matriz más grande) como se describe en la siguiente ecuación:

$$
g(x,y)=w*f(x,y)={1\over c}\sum_{dx=-a}^{a}\sum_{dy=-b}^{b}w(dx,dy)f(x+dx,y+dy)
$$

Donde $f(x,y)$ es la imagen original, $w$ el kernel de filtrado y $g(x,y)$ la imagen resultante del proceso de convolución, $a$ y $b$ representan la distancia desde el elemento central del kernel hacia los bordes, de modo que definen el tamaño del kernel, en este trabajo usaremos una matriz de $3\times3$, por lo que $dx$ y $dy$ tomarán valores discretos en el rango $[-1,1]$ en este caso. $c$ es un factor de control (normalmente la suma de los elementos del kernel), que nos permitirá mantener la proporción del nivel del color resultante en un rango aceptable.

Esta será la ecuación que desarrollaremos de manera funcional, para luego comprobar su funcionamiento con algunas matrices kernel conocidas para evaluar los resultados que obtengamos.

Para esto usaremos las siguientes bibliotecas que importamos a continuación: Pillow, numpy, matplotlib y también usaremos algunas funciones de functools.


In [2]:
import matplotlib.image as img
from PIL import Image
from functools import reduce
import numpy as np
import os

## Definición de funciones

Uno de los principios de la programación funcional consiste en crear funciones que se asemejen a funciones matemáticas, esto es, que sus salidas sean una transformación de sus entradas sin causar modificaciones en otras partes del programa. Para esto, usaremos las funciones `lambda` de python, que ayudarán a mantener una buena legibilidad.

Como la matriz de la imagen contiene vectores de tres elementos, tendremos que definir una función para realizar esta multiplicación, usando funciones lambda tendremos:

In [68]:
# multiplicación de un entero y una tupla
mult_int_trip = lambda i, a, b, c : (i*a, i*b, i*c)

Como vemos, esta función toma cuatro argumentos y los devuelve transformados, sin realizar asignaciones de variables (otro principio de la programación funcional).

Para evitarnos la necesidad de hacer asignaciones de variables, haremos uso en muchos casos de una característica de python: *unpacking* que consiste en realizar asignaciones a tuplas, de modo que se pueden hacer asignaciones de varios valores a la vez o incluso recibir en una tupla los valores de una función que devuelva más de un valor. Pero dijimos que no usaríamos asignaciones, así que usaremos otro aspecto del *unpacking*, en este caso al llamar a funciones, en lugar de llamar a la función `mult_int_trip` con cuatro argumentos, lo haremos con dos: un entero y una tupla de tres elementos, para esto, llamaremos a la función con la siguiente sintaxis:
```
mult_int_trip(i, *t)
```
Donde `i` es un entero y `t` una tupla, nótese que el símbolo de asterisco (`*`) delante de `t` nos permite mapear los tres valores de la tupla `t` en las variables `a`, `b` y `c` de la función, de no incluirlo, tendríamos errores en la ejecución, ya que la función espera cuatro argumentos y estaríamos enviando sólo dos.

De forma similar a la anterior, definiremos otras funciones que requeriremos en el futuro.

In [ ]:
# división de una tupla por un entero
div_trip_int = lambda i, a, b, c : (a/i, b/i, c/i)

# suma de un entero a los elementos de una tupla
sum_trip_int = lambda i, a, b, c : (a+i, b+i, c+i)

# suma de dos tuplas
sum_triplets_tuples = lambda a, b : (a[0]+b[0], a[1]+b[1], a[2]+b[2])

zipwith_tuples = lambda f, a, b : [f(x, *y) for (x, y) in zip(a, b)]

## Iteración

Funciones lambda, unpacking, functools
- zipwith
- operaciones con tuplas (tripletas)
- inbounds
- get surroundings

In [65]:


ib = lambda x : 0 if x < 0 else 255 if x > 255 else x

in_bounds = lambda x, y, z : (ib(x), ib(y), ib(z))

def convolution(arri, arrt, factor, offset, func):
    trip = reduce(sum_triplets_tuples, zipwith_tuples(func, arri, arrt))
    trip = div_trip_int(factor, *trip)
    trip = sum_trip_int(offset, *trip)
    # return sum_trip_int(offset, *div_trip_int(factor, *reduce(sum_triplets_tuples, zipwith_tuples(func, arri, arrt))))

get_surroundings = lambda image, row, col : \
          [image[row-1][col-1],image[row][col-1],image[row+1][col-1],\
           image[row-1][col],image[row][col],image[row+1][col],\
           image[row-1][col+1],image[row][col+1],image[row+1][col+1]]

def bnw (i, trip):
    (a, b, c) = trip
    med = (a+b+c)/3
    return (255-a, 255-b, 255-c)
    return (a, a, a)
    return (med, med, med)

Convolución
- función make_convolution
- Preparación de las funciones que usaremos
- cargado de la imagen


In [66]:
image = img.imread("Downloads/me.jpg")
(height, width, dat) = image.shape

In [67]:
blur_gaussiano = [1, 2, 1, \
                  2, 4, 2, \
                  1, 2, 1]

conv_arr = [-1, 0, -1, \
             0, 4,  0, \
            -1, 0, -1]
factor = 1
offset = 0

image2 = [[[0,0,0] for j in range(width)] for i in range(height)]
image = image.tolist()

from datetime import datetime
time = datetime.now()

def loop(i, j):
     for a in range(i):
         for b in range(j):
             yield (a, b)
            
iterate = lambda x, y : x+y
loop.map(iterate)

for row in range(1, height-1):
  for col in range(1, width-1):
    arr = get_surroundings(image, row, col)
    arrt = convolution(conv_arr, arr, factor, offset, mult_int_trip)
    image2[row][col] = in_bounds(*arrt)
    
print(datetime.now()-time)

0:01:06.164424


In [16]:
i = 100
j = 200

def loop(i, j):
     for a in range(i):
         for b in range(j):
             yield (a, b)
            
iterate = lambda x : x[0]+x[1]
out = map(iterate, loop(i, j))
print(list(out))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30

In [50]:
image2_arr = np.asarray(image2, dtype="uint8")
img = Image.fromarray(image2_arr, 'RGB')
img.show()
img.save("Downloads/imageswwweird.jpg")

In [17]:
# iteracion

concat = lambda x, y : x + y
reduce(concat, [[(x, y) for y in range(20)] for x in range(30)])

def loop(i, j):
     for a in range(i):
         for b in range(j):
             yield (a, b)

In [14]:
map?
